For the Ronbrock method, we need to solve a linear system of the form
$$
M_{ij}x_{j}=b_{i} \;,
$$
with M a square matrix (repeated indecies imply summation).

Such systems are soved by (among  other methods) the so-called LU factorization (or decomposition),
where you decompose $M_{ij}=L_{ik}U_{kj}$ with $L_{i, j>i}=0$, $L_{i, j=i}=1$, $U_{i,j<i}=0$.


That is if $M$ is $N \times N$ matrix, L,U are defined as
\begin{align}
    &L=\left( \begin{matrix}
    1 & 0 & 0 & 0 & \dots &0 & 0 \\
    L_{2,1} & 1 & 0 & 0 & \dots &0 & 0\\
    L_{3,1} & L_{3,2} & 1 & 0 & \dots &0 & 0 \\
    \vdots & \vdots & \vdots & \ddots & \dots & \vdots & \vdots \\
    L_{N-1, 1} & L_{N-1, 2} & L_{N-1, 3} & L_{N-1, 4} & \dots & 1 & 0 \\
    L_{N, 1} & L_{N, 2} & L_{N, 3} & L_{N, 4} & \dots & L_{N, N-1} & 1 \\
    \end{matrix}\right) \;, \\
    %
    &U=\left( \begin{matrix}
    U_{1,1} & U_{1,2} & U_{1,3} & U_{1,4} & \dots & U_{1,N-1} & U_{1,N} \\
    0 & U_{2,2} & U_{2,3} & U_{2,4} & \dots & U_{2,N-1} & U_{2,N}\\
    0 & 0 & U_{3,3} & U_{3,4} & \dots & U_{3,N-1} & U_{3,N} \\
    \vdots & \vdots & \vdots & \ddots & \dots & \vdots & \vdots \\
    0 & 0 & 0 &0 & \dots & U_{N-1,N-1} & U_{N-1,N} \\
    0 & 0 & 0& 0 & \dots & 0 &U_{N,N} \\
    \end{matrix}\right)
    %
\end{align}

Then we have in general $M_{i, j} = \sum_{k=1}^{i}L_{i,k}U_{k,j}$. Since 
$L_{i, k \geq i}=0$ and $U_{k>j,j}=0$, the sum runs up to $j$ if $i \geq j$ and
$i$ if $i \leq j$  (for $i=j$ then both are correct). That is




$$
M_{i, j \geq i} = \sum_{k=1}^{i-1}L_{i,k}U_{k,j}+ U_{i,j} \Rightarrow 
U_{i,j }=M_{i,j } - \sum_{k=1}^{i-1}L_{i,k}U_{k,j }\; , \;\;\; j \geq i \\[0.5cm]
M_{i, j \leq i} = \sum_{k=1}^{j-1}L_{i,k}U_{k,j} +L_{i,j}U_{j,j} \Rightarrow 
L_{i,j}=\left( M_{i,j} - \sum_{k=1}^{j-1}L_{i,k}U_{k,j} \right) U_{jj}^{-1} , \;\;\; j \leq i
$$

Since $U$ and $L$ are triangular matrices, we can solve these two systems sequentially
$$
L_{i,k}y_{k}=b_{k} \\
U_{k,j}x_{j}=y_{k},
$$

since 
$$
y_1 = b_{1} \\
L_{2,1}y_{1}+y_{2}=b_{2} \Rightarrow y_{2}=b_{2}-L_{2,1}y_{1} \\
\vdots \\
y_{i}=b_{i} - \sum_{j=1}^{i-1}L_{i,j}y_{j}
$$

and

$$
U_{N,N}x_{N}=y_{N} \Rightarrow x_{N}=y_{N}/U_{N,N}\\
U_{N-1,N}x_{N}+U_{N-1,N-1}x_{N-1}=y_{N-1} \Rightarrow x_{N-1}=\left(y_{N-1} -U_{N-1,N}x_{N} \right)/U_{N-1,N-1} \\
\vdots \\
x_{i}=\dfrac{y_{i} -\displaystyle\sum_{j=i+1}^{N} U_{i,j}x_{j}  }{U_{i,i}}
$$

Since $x_{i} \sim 1/U_{i,i} =1/M_{i,i}$, if the diagonal terms of $M$ are small (or god forbid they vanish), we would have a problem.


To solve this problem we do $PLU$ decomposition, where $P^{T}M=LU$ with $P$ a permutation matrix so that the diagonals
in $P^{T}M$ contain the largest value of each row.

Then solving $M x =b$ is equavalent to solving $\left( P^{T}M \right) x =P^{T}b$ with LU decomposition of 
$P^{T}M$.


In [1]:
from scipy.linalg import lu_factor,lu_solve,lu

import numpy as np

In [51]:
def max_in(row,start,N):
    _in=start
    _max=row[_in]
    for i in range(_in+1,N):
        if row[i]>_max:
            _max=row[i]
            _in=i
        
            
    return _max,_in

def min_in(row,start,N):
    _in=start
    _min=row[_in]
    for i in range(_in+1,N):
        if row[i]<_min:
            _min=row[i]
            _in=i
    return _min,_in

def Pivot(M,P,N):
    
    for i in range(N):
        for j in range(N):
            if i==j:
                P[i][i]=1
            else:
                P[i][j]=0
    
    
    #have a look at https://www.cs.bgu.ac.il/~mma172/wiki.files/LectureNotes.pdf
    #and http://lampx.tugraz.at/~hadley/num/ch2/2.3a.php
    #It seems that the pivoting is done according to U (which makes sence)
    for col in range(N):
        #fine the maximum and it's index of the column
    
        _tmp=[np.abs(M[r][col]) for r in  range(N)]
        _maxR,_in =max_in(_tmp,0,N)
        
        #print(_tmp)
        #print(_in)
        #Move M[_in][:]  so that its maximum is at M[col][col]  
        #exchange the rows M[col][:] with M[_in][:] 
        #this change should only happen if the current diagonal term |M[_in][_in]| is
        #smaller than the term that will become the diagonal term |M[_in][_in]| if the exhange occurs
        #or that the minimum is not going to to become a diagonal term
        if _maxR>np.abs(M[col][col]): 
            _tmp=[M[col][r] for r in  range(N)]
            _min,m_in =min_in(np.abs(_tmp),0,N)
            #print(col,m_in)
            if  m_in!=_in or np.abs(M[_in][_in])<np.abs(M[col ][_in]):
            #change the pivot matrix:
            #-----------TO DO---------------#
                P_tmp=[P[col][r] for r in  range(N)]
                for r in range(N):
                    M[col][r]=M[_in][r]
                    M[_in][r]=_tmp[r]
                    P[col][r]=P[_in][r]
                    P[_in][r]=P_tmp[r]
                #print(col,_in,np.array(M))
                #print('----------')
            


In [76]:
[ [M0[i][j] for j in range(N)]for i in range(N)]

[[0.0, 3.543325575745879, -7.609186860702108],
 [-5.500949116054825, 5.296773948983242, 7.4971048689821025],
 [0.36842495042955825, 0.24366669121845064, 0.0]]

In [81]:
N=3
M=np.random.rand(N,N)*20-10
M[0][0]=0
M[2][2]=0
# M[2][3]=0
# M[3][3]=0
# M[4][4]=0
# M[3][4]=0
# M[2][4]=0
# M=[[1 , 3,  5],
#   [2  ,4 , 7],
#   [1  ,1 , 0]]

M0=M.copy()

P=[ [0 for j in range(N)] for i in range(N)]
print(M)
print('----------')
print('----------')
print('----------')
Pivot(M,P,N)
print(M)
print('----------')
print('----------')
print([M[i][i] for i in range(N)])#check the diagonals

print('----------')
print('----------')
print(
    np.dot(np.array(P).T,M)-np.array(M0)#check the pivot
)
np.array(P)

[[ 0.         -7.23078462  2.48392358]
 [-6.62897206 -5.17424843  2.23896065]
 [-0.82789109 -6.54476777  0.        ]]
----------
----------
----------
[[-6.62897206 -5.17424843  2.23896065]
 [-0.82789109 -6.54476777  0.        ]
 [ 0.         -7.23078462  2.48392358]]
----------
----------
[-6.628972059877358, -6.544767765796515, 2.4839235823526593]
----------
----------
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


array([[0, 1, 0],
       [0, 0, 1],
       [1, 0, 0]])

In [85]:
M=M0.copy()
P,L,U=lu(M)
print([np.dot(P,np.dot(L,U))[i][i] for i in range(N)])

np.dot(P,np.dot(L,U))

[0.0, -5.174248432765993, 0.0]


array([[ 0.        , -7.23078462,  2.48392358],
       [-6.62897206, -5.17424843,  2.23896065],
       [-0.82789109, -6.54476777,  0.        ]])

In [75]:
def Sum(L,U,i,j,N):
    #this is the sum I'll need for LU
    s=0
    #range(N) goes up to N-1, so in order to call this function as e.g. Sum(L,U,i,j,j-1), we need range(N+1)
    for k in range(N+1):
        s+=L[i][k]*U[k][j]
    return s

def LU(M,N,L,U):
     
    for i in range(N):
        L[i][i]=1
   
    for j in range(N):
        for i in range(N):
            if j>=i:
                U[i][j]=M[i][j]-Sum(L,U,i,j,i-1)
            if j<i:
                L[i][j]=( M[i][j]-Sum(L,U,i,j,j-1) )/U[j][j]

            
        

In [243]:
N=3
M=[
    [9,np.random.rand(),np.random.rand()],
    [np.random.rand(),2,np.random.rand()],
    [np.random.rand(),np.random.rand(),3]
]
L=[ [0  for i in range(N)]  for j in range(N)]
U=[ [0  for i in range(N)]  for j in range(N)]
LU(M,N,L,U)

In [244]:
np.dot(L,U)-np.array(M)

array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])